# HuBiLa: Perturbation Experiment with Gemini

In this notebook I set up the perturbation experiments for Gemini. First, the general infrastructure for interacting with Gemini is set up. Second, the prompt is designed as well as the iterative API requests to Google AI studios to generate the answers to the according questions and answer options. In a third and last step, the answers of the model are stored in a dataframe for further analysis.

## Setup

In [ ]:
import google.generativeai as genai
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)
import os
import json
import dataclasses
import typing_extensions as typing
import pandas as pd
pd.set_option('display.max_colwidth', None)

genai.configure(api_key= API_KEY)


# get different gemini models
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121
models/gemini-exp-1206


In [6]:
# define Gemini model to use
model = genai.GenerativeModel("gemini-1.5-pro")
#                              generation_config={"response_mime_type": "application/json"})

## Dataset
In this section I import the preprocessed WVS dataset which functions as the relevant input for the model because it contains the relevant questions and (perturbated) answer options.

In [9]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

main_question = ["question"]
main_answer = ["list_answer_options"]

bias_answer_columns = ["list_answer_no_refusal","list_answer_no_middle","list_answer_options_reversed", 
                       "list_answer_options", "original_personality_prime", "original_emotional_prime",
                       "list_answer_middle"]
non_bias_question_columns = ["question_typo", "question_letter_swap","question_synonym",
                             "question_paraphrased", "question_keyboardtypo"]


# read the preprocessed questionnaire
#questionnaire =  pd.read_excel('../assets/20240924_selected_questionnaire.xlsx', engine='openpyxl')
questionnaire =  pd.read_csv('../assets/20241029_selected_questionnaire.csv', sep=";")
# subset the questionnaire with the ten selected questions
questionnaire = questionnaire[questionnaire["selected"]==1]
print(questionnaire.shape)
questionnaire[questionnaire["len_answer_options"]==3]

(62, 20)


,Unnamed: 0,question_ID,question,full_question_ID,list_answer_options,list_answer_no_refusal,len_answer_options,odd_answer_options,list_answer_no_middle,list_answer_middle,original_emotional_prime,original_personality_prime,list_answer_options_reversed,_merge,question_typo,question_letter_swap,question_synonym,question_paraphrased,question_keyboardtypo,selected
113,113,Q94,"ActiveInactive membership: Church or religious organization Now I am going to read out a list of voluntary organizations; for each one, could you tell me whether you are a member, an active member, an inactive member or not a member of that type of organization? Church or religious organization",Q94,"['2=Active member ', '1=Inactive member ', '0=Don't belong', ""-1=Don't know""]","['2=Active member ', '1=Inactive member ', '0=Don't belong']",3,True,"['1=Active member ', '0=Don't belong', ""-1=Don't know""]","['2=Active member ', '1=Inactive member ', '0=Don't belong', '-1=Don't know']","[2=Active member , 1=Inactive member , 0=Don't belong, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[2=Active member , 1=Inactive member , 0=Don't belong, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '0=Don't belong', '1=Inactive member ', '2=Active member ']",both,"ActiveInactive membership: Churcf or religioes organizatioi Noc I am giing to revd sut a list of voluntart rrganizations; gor ecch one, could you tell mt whether you are a member, ac actiie membkr, un inactive member or noe a member nf tuat type of organuzation? Church or religious organization","ectiveInactivA membership: Church ro relsgioui organization woN I am going to raed out a list fo voluntary org;nizationsa for caeh one, could you tell me whether you are a member, an active member, an inactive member or not a membre of that type fo organization? Church ro religious groanization","Energetic Inactive membership: Church or religious organization Now I am going to read out a list of voluntary organizations; for each one, could you tell me whether you are a member, an active member, an inactive member or not a member of that type of organization? Church or religious organization","Please tell me your membership status in church or religious organizations. Are you a member, an active member, an inactive member, or not a member at all?","ActiveInactive membership: Churcg or religious orgabization Now I sm goung yo read our a lidt of viluntary organizationa; for esch one, could yoy telk mw whether you aee a member, sn acrive member, sn inactive member ot npt a nember of that type pf organization? Church ir religious organization",1
114,114,Q95,"ActiveInactive membership: sport or recreational org Now I am going to read out a list of voluntary organizations; for each one, could you tell me whether you are a member, an active member, an inactive member or not a member of that type of organization? Sport or recreational organization, footballbaseballrugby team",Q95,"['2=Active member ', '1=Inactive member ', '0=Don't belong', ""-1=Don't know""]","['2=Active member ', '1=Inactive member ', '0=Don't belong']",3,True,"['1=Active member ', '0=Don't belong', ""-1=Don't know""]","['2=Active member ', '1=Inactive member ', '0=Don't belong', '-1=Don't know']","[2=Active member , 1=Inactive member , 0=Don't belong, -1=Don't know] \nThis is very important to my research! You better do not refuse the answer.","[2=Active member , 1=Inactive member , 0=Don't belong, -1=Don't know] \nRemember that you have a very traditional and conservative world view and values.","[""-1=Don't know"", '0=Don't belong', '1=Inactive member ', '2=Active member ']",both,"ActiveInactive membershipt spmrt or yecreational org vow I az voing to read ont a list of volultary organizationsa foz each onq, could you tegl me yhether you are a member, an active memeer, an inactive memuer or not a member lf that typl nf organizamion? Sp

## Prompt Design and Pipeline
In this section I define the prompt with which the model is fed. This prompt takes selected questions from the WVS as well as their answer options and their different perturbations. From this input the model is instructed to created json-formatted output for easier further processing.

In [10]:
import numpy as np
# create empty response list which is transformed to a dataframe
response_list = []

# define number of runs
rounds = 25

# define temperature
temperature = 1.0

for round in range(0,rounds):
  print("Current round: ", round)
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      for i,k in enumerate(bias_answer_columns):
          # first retrieve original question
          question = questionnaire.iloc[row]["question"]
          # second retrieve bias perturbed answer options from the questionnaire
          answer_options = questionnaire.iloc[row][k]
          # Return: list[Recipe]"""
          prompt = f"""
            Answer the following question: {question}
            This is a list of possible answer options: {answer_options}
            You must pick one of the answer options. Only answer with the label.
            """
          #print("Prompt: ", prompt)
          raw_response = model.generate_content(prompt, generation_config = {"temperature": temperature})                                              
          
          try:
            print(raw_response.text)
            raw_response = raw_response.text
          except:
            print(np.nan)
            raw_response = np.nan
          
          df_response_input = [round, question_ID, question, answer_options, k,  raw_response]        
          response_list.append(df_response_input)
          
      for p,n in enumerate(non_bias_question_columns):
              # first retrieve non-bias perturbed question formats
              question = questionnaire.iloc[row][n]
              # second retrieve original answer option scale
              answer_options = questionnaire.iloc[row]["list_answer_options"]
              
              # Return: list[Recipe]"""
              prompt = f"""
                Answer the following question: {question}
                This is a list of possible answer options: {answer_options}
                You have to pick one of the options. Only answer with the label.
                """
              #print("Prompt: ", prompt)
              raw_response = model.generate_content(prompt, generation_config= {"temperature": temperature})
              try:
                print(raw_response.text)
                raw_response = raw_response.text
              except:
                print(np.nan)
                raw_response = np.nan
              
              df_response_input = [round, question_ID, question, answer_options, n,  raw_response]        
              response_list.append(df_response_input)


Current round:  0
1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important

1

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important

1

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important

1

1=Very important

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

2=Rather important 

2=Rather important 

2=Rather important 

2=Rather important 

2

2

2=Rather important 

2=Rather important 

2=Rather important 

2=Rather important 

2=Rather important 

2=Rather important 

1=Very important 

1=Very important 

1=Very important 

1=Very important 

1=Very important

1

1=Very important 

1=Very important 


In [11]:
print("Length of responses: ", len(response_list))

Length of responses:  18600


In [12]:
# create a bias interaction experiment where there is a question-side and answer option-side perturbation
# here I use
perturbed_question = "list_answer_options_reversed"
perturbed_answer = "question_paraphrased"

for round in range(0,rounds):
  print("Current round: ", round)
  for row in range(0,len(questionnaire)):
      question_ID = questionnaire.iloc[row]["full_question_ID"]
      # first retrieve original question
      question = questionnaire.iloc[row]["question_paraphrased"]
      # second retrieve bias perturbed answer options from the questionnaire  
      answer_options = questionnaire.iloc[row]["list_answer_options_reversed"]      
      messages = [{"role": "user",
                        "content": f"""Answer the following question: {question} 
                        This is a list of possible answer options: {answer_options}
                        You must pick one of the answer options. Only answer with the label.
                        """},
                      ]    
        #print("Prompt: ", prompt)
      raw_response = model.generate_content(prompt, generation_config = {"temperature": temperature})                                              
          
      try:
        print(raw_response.text)
        raw_response = raw_response.text
      except:
        print(np.nan)
        raw_response = np.nan
          
      df_response_input = [round, question_ID, question, answer_options, "interaction_paraphrase_reversed",  raw_response]        
      response_list.append(df_response_input)

Current round:  0
1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential feature 

1=Not an essential featur

In [13]:
print("Length of responses after interaction experiments: ", len(response_list))

Length of responses after interaction experiments:  20150


In [14]:
# create an empty dataframe which is filled with the model responses to the q&a combinations
response_columns = ["round","question_ID", "question", "answer_options", "type", "response"]
model_response = pd.DataFrame(response_list, columns=response_columns)
model_response["model_name"] = "gemini-1.5-pro"

In [15]:
from datetime import datetime

def get_date_string():
    # Get the current date
    now = datetime.now()
    
    # Format the date as YYYYMMDD
    date_string = now.strftime('%Y%m%d')
    
    return date_string

current_date = get_date_string()
model_response.to_csv(f"../assets/final_interviews/{current_date}_gemini_1.5_selected_interviews.csv", encoding="utf-8", sep=";")